In [1]:
import numpy as np
import itertools
import scipy.spatial
import skimage.io as io
from skimage.feature import blob_log, blob_doh, blob_dog
from scipy import ndimage, spatial
import matplotlib.pyplot as plt
import star
from PIL import Image

C:\Users\panf\AppData\Roaming\Python\Python38\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
def proposecands(uknquadlist, refquadlist, n=5, verbose=True):
    """
    Function that identifies similar quads between the unknown image and a reference.
    Returns a dict of (uknquad, refquad, dist, trans)
    """
    # Nothing to do if the quadlists are empty ...
    if len(uknquadlist) == 0 or len(refquadlist) == 0:
        if verbose:
            print("No quads to propose ...")
        return []

    if verbose:
        print("Finding %i best candidates among %i x %i (ukn x ref)" % (n, len(uknquadlist), len(refquadlist)))
    uknhashs = np.array([q.hash for q in uknquadlist])
    refhashs = np.array([q.hash for q in refquadlist])

    # Brute force...
    dists = scipy.spatial.distance.cdist(refhashs, uknhashs)
    uknmindistindexes = np.argmin(dists, axis=0)  # For each ukn, the index of the closest ref
    uknmindist = np.min(dists, axis=0)  # The corresponding distances
    uknbestindexes = np.argsort(uknmindist)

    candlist = []
    nmax = len(uknbestindexes)
    if verbose:
        print("We have a maximum of %i quad pairs" % (nmax))
    for i in range(min(n, nmax)):

        cand = {"uknquad": uknquadlist[uknbestindexes[i]], "refquad": refquadlist[uknmindistindexes[uknbestindexes[i]]],
                "dist": uknmindist[uknbestindexes[i]]}

        cand["trans"] = quadtrans(cand["uknquad"], cand["refquad"])
        print ((abs(cand["trans"].getrotation())), cand["dist"]) 
        if (abs(cand["trans"].getrotation()) < 5): 
             candlist.append(cand)
             if verbose:
                 print("Cand %2i (dist. %12.8f) : %s" % (i+1, cand["dist"], str(cand["trans"])))

    return candlist

In [3]:

def blob_detection(img_path, min_sigma, max_sigma, threshold, method=0):
    """This function is mostly used for detecting the beads in any image.

    Args:
        img_path (string): The absolute path of the input image.
        min_sigma (int): The minimum sigma, lower it is, smaller the blob will be detected.
        max_sigma (int): The maximum sigma, higher it is, bigger the blob will be detected.
        threshold (float): Higher it is, higher the intensities of blobs.
        method (int, optional): 0 for Difference of Gaussian (DoG) and 1 for Determinant of Hessian (DoH). 
        They should be applied with different combination of parameters. DoG is more suitable for fret movies,
        while DoH is more suitable for sequencing images. Defaults to 0.

    Returns:
        centers: A numpy array containing the coordinates of all the centers.
    """
    img = io.imread(img_path)
    if method == 0:
        blob = blob_dog(
            img, min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold
        )
    else:
        blob = blob_doh(
            img, min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold
        )
    i = 0
    # r = 3
    centers = []
    h, w = img.shape
    for blob in blob:
        y, x, r = blob
        if y > r and y < (h - r) and x > r and x < (w - r):
            centers.append(
                ndimage.measurements.center_of_mass(
                    img[int(y - r) : int(y + r + 1), int(x - r) : int(x + r + 1)]
                )
            )
            centers[i] = list(np.add(centers[i], [x - r, y - r]))
            i += 1
    
    return np.array(centers)
 



def show_blob_detection_res(img_path, min_sigma, max_sigma, threshold, method=0):
    """
    Showing the result of 'blob detection' function. Used as the same way of 'blob_detection'
    """
    fig, ax = plt.subplots()
    img = io.imread(img_path)
    ax.imshow(img)
    if method == 0:
        res = blob_dog(
            img,
            min_sigma=min_sigma,
            max_sigma=max_sigma,
            threshold=threshold,
        )
    else:
        res = blob_doh(
            img,
            min_sigma=min_sigma,
            max_sigma=max_sigma,
            threshold=threshold,
        )
    i = 0
    CM = []
    r = 3
    [h, w] = img.shape
    for blob in res:
        y, x, r = blob
        # print(r)
        if y > r and y < (h - r) and x > r and x < (w - r):
            CM.append(
                ndimage.measurements.center_of_mass(
                    img[int(y - r) : int(y + r), int(x - r) : int(x + r)]
                )
            )
            CM[i] = list(np.add(CM[i], [y - r, x - r]))
            x1, y1 = CM[i]
            c = plt.Circle([y1, x1], 3, color="red", linewidth=1, fill=False)
            ax.add_patch(c)
            i += 1
    ax.set_axis_off()
    plt.show()
    
def show_blob(img_path, centers):
    """
    Showing the result of 'blob detection' function. Used as the same way of 'blob_detection'
    """
    fig, ax = plt.subplots()
    img = io.imread(img_path)
    ax.imshow(img)   
    for row in centers:
        x1, y1 = row 
        # print(r)
        c = plt.Circle([x1, y1], 3, color="red", linewidth=1, fill=False)
        ax.add_patch(c)
         
    ax.set_axis_off()
    plt.show()    

In [4]:
"""
Overhaul of cosmouline's star module, for alipy2.
This module contains stuff for geometric matching algorithms.
"""

import sys
import os
import math
import numpy as np
import operator  # For sorting
import copy
import scipy.linalg
import scipy.spatial


class Dots:
    """
    Simple class to represent a single source (usually stars, but not necessarily).
    In this module we often manipulate lists of such Dots objects.
    """

    def __init__(self, x=0.0, y=0.0, props={}, fwhm=-1.0, elon=-1.0):
        """
        flux : Some "default" or "automatic" flux, might be a just good guess. Used for sorting etc.
        If you have several fluxes, colours, store them in the props dict.
        props : A placeholder dict to contain other properties of your choice (not required nor used by the methods).
        """
        self.x = float(x)
        self.y = float(y)
        
       
    def copy(self):
        return copy.deepcopy(self)

    def __getitem__(self, key):
        """
        Used for sorting list of stars.
        """
        if key == 'flux':
            return self.flux
        if key == 'fwhm':
            return self.fwhm
        if key == 'elon':
            return self.elon

  
    
    def coords(self, full=False):
        """
        Returns the coords in form of an array.

        :param full: If True, I also include flux, fwhm, elon
        :type full: boolean

        """
        if full:
            return np.array([self.x, self.y, self.flux, self.fwhm, self.elon])
        else:
            return np.array([self.x, self.y])

    def distance(self, otherstar):
        """
        Returns the distance between the two stars.
        """
        return math.sqrt(np.sum((self.coords() - otherstar.coords())**2))

    def distanceandsort(self, otherstarlist):
        """
        Returns a list of dicts(star, dist, origpos), sorted by distance to self.
        The 0th star is the closest.

        otherstarlist is not modified.
        """

        returnlist = []
        for i, star in enumerate(otherstarlist):
            dist = self.distance(star)
            returnlist.append({'dot': star, 'dist': dist, 'origpos': i})
        returnlist = sorted(returnlist, key=operator.itemgetter('dist'))  # sort stars according to dist

        return returnlist

# And now some functions to manipulate list of such stars ###


def listtoarray(starlist, full=False):
    """
    Transforms the starlist into a 2D numpy array for fast manipulations.
    First index is star, second index is x or y

    :param full: If True, I also include flux, fwhm, elon
    :type full: boolean

    """
    return np.array([star.coords(full=full) for star in starlist])


def area(dotlist, border=0.01):
    """
    Returns the area covered by the dots.
    Border is relative to max-min
    """
    if len(dotlist) == 0:
        return np.array([0, 1, 0, 1])

    if len(dotlist) == 1:
        dot = dotlist[0]
        return np.array([dot.x - 0.5, dot.x + 0.5, dot.y - 0.5, dot.y + 0.5])

    a = listtoarray(starlist)
    (xmin, xmax) = (np.min(a[:, 0]), np.max(a[:, 0]))
    (ymin, ymax) = (np.min(a[:, 1]), np.max(a[:, 1]))
    xw = xmax - xmin
    yw = ymax - ymin
    xmin = xmin - border*xw
    xmax = xmax + border*xw
    ymin = ymin - border*yw
    ymax = ymax + border*yw
    return np.array([xmin, xmax, ymin, ymax])





In [5]:
class Quad:
    """
    A geometric "hash", or asterism, as used in Astrometry.net :
    http://adsabs.harvard.edu/cgi-bin/bib_query?arXiv:0910.2233
    It is made out of 4 stars, and it is shift / scale / rotation invariant
    """

    def __init__(self, fourstars):
        """
        fourstars is a list of four stars

        We make the following attributes :
        self.hash
        self.stars (in the order A, B, C, D)

        """
        assert len(fourstars) == 4

        tests = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
        other = [(2, 3), (1, 3), (1, 2), (0, 3), (0, 2), (0, 1)]
        dists = np.array([fourstars[i].distance(fourstars[j]) for (i, j) in tests])
        assert np.min(dists) > 1.0

        maxindex = np.argmax(dists) # Returns the indices of the maximum values along an axis
        (Ai, Bi) = tests[maxindex]  # Indexes of stars A and B
        (Ci, Di) = other[maxindex]  # Indexes of stars C and D
        
        A = fourstars[Ai]
        B = fourstars[Bi]
        C = fourstars[Ci]
        D = fourstars[Di]

        # We look for matrix transform [[a -b], [b a]] + [c d] that brings A and B to 00 11 :
        
        
        x = B.x - A.x
        if x == 0:
            x = x+0.01
        y = B.y - A.y
        b = (x-y)/(x*x + y*y)
        a = (1.0/x) * (1.0 + b*y)
        c = b*A.y - a*A.x
        d = - (b*A.x + a*A.y)

        t = star.SimpleTransform((a, b, c, d))

        (xC, yC) = t.apply(C.x, C.y)
        (xD, yD) = t.apply(D.x, D.y)

        # Normal case
        self.hash = (xC, yC, xD, yD)

        # Break symmetries :
        testa = xC > xD
        testb = xC + xD > 1

        if testa and not testb:  # we switch C and D
            self.hash = (xD, yD, xC, yC)
            (C, D) = (D, C)

        if testb and not testa:  # We switch A and B
            self.hash = (1.0-xD, 1.0-yD, 1.0-xC, 1.0-yC)
            (A, B) = (B, A)
            (C, D) = (D, C)

        if testa and testb:
            self.hash = (1.0-xC, 1.0-yC, 1.0-xD, 1.0-yD)
            (A, B) = (B, A)

        # Checks :
        assert self.hash[0] <= self.hash[2]
        assert self.hash[0] + self.hash[2] <= 1

        self.stars = [A, B, C, D]  # Order might be different from the fourstars !

    def __str__(self):
        return "Hash : %6.3f %6.3f %6.3f %6.3f " % (
            self.hash[0], self.hash[1], self.hash[2], self.hash[3])


In [ ]:
seq = io.imread(current_direct+"/MIN_seq_stack.tif")
seq = seq.astype("ushort")
img1 = io.imread(current_direct + "/Pos154.tiff") #reading the stack of images
img1 = img1.astype("ushort") #turn from float format to ushort
img1 = img1 [256:512, 0:512] 
img2 = Image.fromarray(img1)
img2.save( 'Pos154_cut.tif')

In [ ]:
seq_centers = blob_detection(
                         current_direct+"/MIN_seq_stack-1.tif",
                         min_sigma=1,
                         max_sigma=10,
                         threshold=0.08,
                         )
show_blob_detection_res(
                        current_direct+"/MIN_seq_stack-1.tif",
                         min_sigma=1,
                         max_sigma=10,
                         threshold=0.05,
                         )
FRET_centers = blob_detection(
                         current_direct + "/Pos154.tiff",
                         min_sigma=1,
                         max_sigma=10,
                         threshold=0.01,
                         )
show_blob_detection_res(
                        current_direct + "/Pos154.tiff",
                         min_sigma=1,
                         max_sigma=10,
                         threshold=0.01,
                         )


In [6]:
def distance(self, otherstar):
        """
        Returns the distance between the two stars.
        """
        return math.sqrt(np.sum((self.coords() - otherstar.coords())**2))

def mindist(fourstars):
    """
    Function that tests if 4 stars are suitable to make a good quad...
    """
    tests = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
    dists = np.array([fourstars[i].distance(fourstars[j]) for (i, j) in tests])
    return np.min(dists)



def makequads1(center_coord, n=10, s=0, d=10, verbose=True):
    """
    First trivial quad maker.
    Makes combis of the n brightest stars.

    :param n: number of stars to consider (brightest ones).
    :type n: int
    :param s: how many of the brightest stars should I skip ?
        This feature is useful to avoid building quads with nearly saturated stars that are not
        available in other exposures.
    :type s: int
    :param d: minimal distance between stars
    :type d: float

    """
    quadlist = []
     

    for fourdots in itertools.combinations(center_coord[s:s+n], 4):
        if mindist(fourdots) > d:
                quadlist.append(Quad(fourdots))

    if verbose:
        print("Made %4i quads from %4i stars (combi n=%i s=%i d=%.1f)" % (len(quadlist), len(center_coord), n, s, d))

    return quadlist


In [ ]:
dotlist = []
for row in FRET_centers: 
    dotlist.append(Dots(row[0], row[1]))
makequads1(dotlist)    

In [7]:
def makequads2(dotlist, f=5.0, n=6, s=0, d=50.0, verbose=True):
    """
    Similar, but fxf in subareas roughly f times smaller than the full frame.
    s allows to skip the brightest stars in each region

    :param f: smallness of the subareas
    :type f: float
    :param n: number of stars to consider in each subarea
    :type n: int
    :param d: minimal distance between stars
    :type d: float
    :param s: number of brightest stars to skip in each subarea
    :type s: int

    """
    quadlist = []
    
    (xmin, xmax, ymin, ymax) = star.area(dotlist)

    r = 2.0 * max(xmax - xmin, ymax - ymin) / f

    for xc in np.linspace(xmin, xmax, int(f)+2)[1:-1]:
        for yc in np.linspace(ymin, ymax, int(f)+2)[1:-1]:
            dot = Dots(x=xc, y=yc)
            das = dot.distanceandsort(dotlist)
            elmnt = [element["dot"] for element in das if element["dist"] <= r]
            brightestwithinr = elmnt[s:s+n]
            for fourstars in itertools.combinations(brightestwithinr, 4):
                if mindist(fourstars) > d:
                    quadlist.append(Quad(fourstars))

    if verbose:
        print("Made %4i quads from %4i stars (combi sub f=%.1f n=%i s=%i d=%.1f)" % (len(quadlist), len(dotlist),
                                                                                     f, n, s, d))
    return quadlist

In [ ]:
makequads2(dotlist)

In [8]:
from os import path
class ImgCat:
    """
    Represent an individual image and its associated catalog, starlist, quads etc.
    """

    def __init__(self, filepath):
        """

        :param filepath: Path to the FITS file, or alternatively just a string to identify the image.
        :type filepath: string

        
        """
        self.filepath = filepath
        print (self.filepath)
        (imgdir, filename) = path.split(filepath)
        (common, ext) = path.splitext(filename)
        self.name = common

       
        self.dotlist = []
        self.mindist = 0.0
        self.xlim = (0.0, 0.0)  # Will be set using the catalog -- no need for the FITS image.
        self.ylim = (0.0, 0.0)

        self.quadlist = []
        self.quadlevel = 0  # encodes what kind of quads have already been computed

    def __str__(self):
        return "%20s: approx %4i x %4i, %4i stars, %4i quads, quadlevel %i" % (path.basename(self.filepath),
                                                                               self.xlim[1] - self.xlim[0],
                                                                               self.ylim[1] - self.ylim[0],
                                                                               len(self.starlist), len(self.quadlist),
                                                                               self.quadlevel)

    """
    def makedotlist(self, skipsaturated=False, n=200, verbose=True):
        if self.cat:
            if skipsaturated:
                maxflag = 3
            else:
                maxflag = 7
            self.starlist = star.sortstarlistbyflux(star.readsexcat(self.cat, hdu=self.hdu, maxflag=maxflag,
                                                                    verbose=verbose))[:n]
            (xmin, xmax, ymin, ymax) = star.area(self.starlist, border=0.01)
            self.xlim = (xmin, xmax)
            self.ylim = (ymin, ymax)

            # Given this starlists, what is a good minimal distance for stars in quads ?
            self.mindist = min(min(xmax - xmin, ymax - ymin) / 10.0, 30.0)
            return 0

        else:
            return 1
      """        
        
    def makedotlist(self, min_sigma, max_sigma, threshold, method=0):
        """
        This function is mostly used for detecting the beads in any image.

        Args:
            img_path (string): The absolute path of the input image.
            min_sigma (int): The minimum sigma, lower it is, smaller the blob will be detected.
            max_sigma (int): The maximum sigma, higher it is, bigger the blob will be detected.
            threshold (float): Higher it is, higher the intensities of blobs.
            method (int, optional): 0 for Difference of Gaussian (DoG) and 1 for Determinant of Hessian (DoH). 
            They should be applied with different combination of parameters. DoG is more suitable for fret movies,
            while DoH is more suitable for sequencing images. Defaults to 0.

        Returns:
            centers: A numpy array containing the coordinates of all the centers.
        """
        img = io.imread(self.filepath)
        print(self.filepath)
        if method == 0:
                blob = blob_dog(
                    img, min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold
                )
        else:
                blob = blob_doh(
                    img, min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold
                )
        i = 0
        # r = 3
        centers = []
        h, w = img.shape
        for blob in blob:
            y, x, r = blob
            if y > r and y < (h - r) and x > r and x < (w - r):
                centers.append(
                   ndimage.measurements.center_of_mass(
                            img[int(y - r) : int(y + r + 1), int(x - r) : int(x + r + 1)]
                        )
                    )
                centers[i] = list(np.add(centers[i], [x - r, y - r]))
                i += 1

        for row in centers: 
            self.dotlist.append(Dots(row[0], row[1]))      
        (xmin, xmax, ymin, ymax) = star.area(self.dotlist, border=0.01)
        self.xlim = (xmin, xmax)
        self.ylim = (ymin, ymax)
    
        return 0
        
       



    def makemorequads(self, verbose=True):
        """
        We add more quads, following the quadlevel.
        """
        # if not add:
        #    self.quadlist = []
        if verbose:
            print("Making more quads, from quadlevel %i ..." % self.quadlevel)
        if self.quadlevel == 0:
            self.quadlist.extend(makequads1(self.dotlist, n=10, d=self.mindist, verbose=verbose))
        elif self.quadlevel == 1:
            self.quadlist.extend(makequads2(self.dotlist, f=3, n=10, d=self.mindist, verbose=verbose))
        elif self.quadlevel == 2:
            self.quadlist.extend(makequads2(self.dotlist, f=6, n=10, d=self.mindist, verbose=verbose))
        elif self.quadlevel == 3:
            self.quadlist.extend(makequads2(self.dotlist, f=12, n=10, d=self.mindist, verbose=verbose))
        elif self.quadlevel == 4:
            self.quadlist.extend(makequads2(self.dotlist, f=15, n=10, s=0, d=self.mindist, verbose=verbose))
        elif self.quadlevel == 5:
            self.quadlist.extend(makequads2(self.dotlist, f=18, n=10, s=0, d=self.mindist, verbose=verbose))
        elif self.quadlevel == 6:
            self.quadlist.extend(makequads2(self.dotlist, f=21, n=10, s=0, d=self.mindist, verbose=verbose))
        elif self.quadlevel == 7:
            self.quadlist.extend(makequads2(self.dotlist, f=24, n=10, s=0, d=self.mindist, verbose=verbose))  
        elif self.quadlevel == 8:
            self.quadlist.extend(makequads2(self.dotlist, f=27, n=10, s=0, d=self.mindist, verbose=verbose))  
        elif self.quadlevel == 9:
            self.quadlist.extend(makequads2(self.dotlist, f=30, n=10, s=0, d=self.mindist, verbose=verbose)) 
        elif self.quadlevel == 10:
            self.quadlist.extend(makequads2(self.dotlist, f=33, n=10, s=0, d=self.mindist, verbose=verbose))   
        elif self.quadlevel == 11:
            self.quadlist.extend(makequads2(self.dotlist, f=36, n=10, s=0, d=self.mindist, verbose=verbose))  
        elif self.quadlevel == 12:
            self.quadlist.extend(makequads2(self.dotlist, f=39, n=10, s=0, d=self.mindist, verbose=verbose))    
        elif self.quadlevel == 13:
            self.quadlist.extend(makequads2(self.dotlist, f=42, n=10, s=0, d=self.mindist, verbose=verbose)) 
        elif self.quadlevel == 14:
            self.quadlist.extend(makequads2(self.dotlist, f=45, n=10, s=0, d=self.mindist, verbose=verbose)) 
        else:
            return False

        self.quadlist = removeduplicates(self.quadlist, verbose=verbose)
        self.quadlevel += 1
        return True

In [9]:
def removeduplicates(quadlist, verbose=True):
    """
    Returns a quadlist without quads with identical hashes...
    """
    # To avoid crash in lexsort if quadlist is too small :
    if len(quadlist) < 2:
        return quadlist
    hasharray = np.array([q.hash for q in quadlist])

    order = np.lexsort(hasharray.T)
    hasharray = hasharray[order]
    diff = np.fabs(np.diff(hasharray, axis=0))
    ui = np.ones(len(hasharray), 'bool')
    ui[1:] = (diff >= 0.000001).any(axis=1)
    if verbose:
        print("Removing %i/%i duplicates" % (len(quadlist) - np.sum(ui), len(quadlist)))

    return [quad for (quad, u) in zip(quadlist, ui) if u]
def quadtrans(uknquad, refquad):
    """
    Quickly return a transform estimated from the stars A and B of two quads.
    """
    return star.fitstars(uknquad.stars[:2], refquad.stars[:2])


In [10]:
class Identification:
    """
    Represents the identification of a transform between two ImgCat objects.
    Regroups all the star catalogs, the transform, the quads, the candidate, etc.

    All instance attributes are listed below.

    :ivar ref: ImgCat object of the reference image
    :ivar ukn: ImgCat object of the unknown image
    :ivar ok: boolean, True if the idendification was successful.
    :ivar trans: The SimpleTransform object that represents the geometrical transform from ukn to ref.
    :ivar uknmatchstars: A list of Star objects of the catalog of the unknown image...
    :ivar refmatchstars: ... that correspond to these Star objects of the reference image.
    """

    def __init__(self, ref, ukn):
        """
        :param ref: The reference image
        :type ref: ImgCat object
        :param ukn: The unknown image, whose transform will be adjusted to match the ref
        :type ukn: ImgCat object
        """
        self.ref = ref
        self.ukn = ukn

        self.ok = False

        self.trans = None
        self.uknmatchstars = []
        self.refmatchstars = []
        self.cand = None

    def findtrans(self, r=5.0, verbose=False):
        """
        Find the best trans given the quads, and tests if the match is sufficient
        """

        # Some robustness checks
        if len(self.ref.dotlist) < 4:
            if verbose:
                print("Not enough dot in the reference catalog.")
            return
        if len(self.ukn.dotlist) < 4:
            if verbose:
                print("Not enough dot in the unknown catalog.")
            return

        # First question : how many stars should match ?
        if len(self.ukn.dotlist) < 5:  # Then we should simply try to get the smallest distance...
            minnident = 4
        else:
            minnident = max(4, min(8, len(self.ukn.dotlist)/5.0))  # Perfectly arbitrary, let's see how it works

        # Hmm, arbitrary for now :
        minquaddist = 0.02

        # Let's start :
        if self.ref.quadlevel == 0:
            self.ref.makemorequads(verbose=verbose)
        if self.ukn.quadlevel == 0:
            self.ukn.makemorequads(verbose=verbose)

        while self.ok is False:
            # Find the best candidates
            cands = proposecands(self.ukn.quadlist, self.ref.quadlist, n=4, verbose=verbose)

            if len(cands) != 0 and cands[0]["dist"] < minquaddist:
                # If no quads are available, we directly try to make more ones.
                for cand in cands:
                    # Check how many stars are identified...
                    nident = star.identify(self.ukn.dotlist, self.ref.dotlist, trans=cand["trans"],
                                           r=r, verbose=verbose, getstars=False)
                    if nident >= minnident:
                        self.trans = cand["trans"]
                        self.cand = cand
                        self.ok = True
                        break  # get out of the for

            if self.ok is False:
                # We add more quads...
                addedmorerefquads = self.ref.makemorequads(verbose=verbose)
                addedmoreuknquads = self.ukn.makemorequads(verbose=verbose)

                if addedmorerefquads is False and addedmoreuknquads is False:
                    break  # get out of the while, we failed.

        #if  ((self.ok) and (abs(self.trans.getrotation()))<5):  # we refine the transform
        if  (self.ok) :
            # get matching stars :
            (self.uknmatchstars, self.refmatchstars) = star.identify(self.ukn.dotlist, self.ref.dotlist,
                                                                     trans=self.trans, r=r, verbose=False,
                                                                     getstars=True)
            # refit the transform on them :
            if verbose:
                print("Refitting transform (before/after) :")
                print(self.trans)
            newtrans = star.fitstars(self.uknmatchstars, self.refmatchstars)
            if newtrans is not None:
                self.trans = newtrans
                if verbose:
                    print(self.trans)
            # Generating final matched star lists :
            (self.uknmatchstars, self.refmatchstars) = star.identify(self.ukn.dotlist, self.ref.dotlist,
                                                                     trans=self.trans, r=r, verbose=verbose,
                                                                     getstars=True)

            if verbose:
                print("I'm done!")

        else:
            if verbose:
                print("Failed to find transform!")

 
def run(ref, ukns, skipsaturated=False, r=5.0, n=500, sexkeepcat=False, sexrerun=True,
        verbose=True, polarMode=None, refpolar=False, camera=None):
    """
    Top-level function to identify transorms between images.
    Returns a list of Identification objects that contain all the info to go further.

    :param ref: path to a FITS image file that will act as the "reference".
    :type ref: string

    :param ukns: list of paths to FITS files to be "aligned" on the reference. **ukn** stands for unknown.
    :type ukns: list of strings

    :param hdu: The hdu of the fits files (same for all) that you want me to use. 0 is somehow "automatic".
    If multihdu, 1 is usually science.

    if the identification fails).

    :param skipsaturated: Should I skip saturated stars ?
    :type skipsaturated: boolean

    :param r: Identification radius in pixels of the reference image (default 5.0 should be fine).
    :type r: float
    :param n: Number of brightest stars of each image to consider (default 500 should be fine).
    :type n: int

    :param sexkeepcat: Put this to True if you want me to keep the SExtractor catalogs (in a dir "alipy_cats").
    :type sexkeepcat: boolean
    :param sexrerun: Put this to False if you want me to check if I have some previously kept catalogs
    (with sexkeepcat=True), instead of running SExtractor again on the images.
    :type sexrerun: boolean
    """

    if verbose:
        print(10*"#", " Preparing reference ...")
    ref = ImgCat(ref)
    retCode = ref.makedotlist(min_sigma=1,
                              max_sigma=10,
                              threshold=0.01)
    
    show_blob_detection_res(
                         path_ref,
                         min_sigma=1,
                         max_sigma=10,
                         threshold=0.01,
                         )
    
    if retCode:
        return None
    ref.makemorequads(verbose=verbose)

    identifications = []

   

    if verbose:
            print(10*"#", "Processing %s" % (ukns))

    ukn = ImgCat(ukns)
        
    ukn.makedotlist( min_sigma=1,
                     max_sigma=10,
                     threshold=0.01)
    
    show_blob_detection_res(
                         path_unk,
                         min_sigma=1,
                         max_sigma=10,
                         threshold=0.01,
                         )

    idn = Identification(ref, ukn)
    idn.findtrans(verbose=verbose, r=r)
    identifications.append(idn)

    return identifications


In [ ]:
current_direct = "C:/Users/panf/Documents/Muscle_intermediates/test_folder_5/Pos126/"
path_ref = current_direct+"Pos126_contr_MIN_seq_stack.tif"
path_unk = current_direct + "Pos126_beads_transformed.tiff"
print(path_unk)
a = run(path_ref, path_unk)

In [ ]:
print(a[0].cand["refquad"]) 

In [11]:
def identify(uknstars, refstars, trans=None, r=5.0, verbose=True, getstars=False):
    """
    Allows to:
     * get the number or matches, i.e. evaluate the quality of the trans
     * get corresponding stars from both lists (without the transform applied)

    :param getstars: If True, I return two lists of corresponding stars, instead of just the number of matching stars
    :type getstars: boolean

    Inspired by the "formpairs" of alipy 1.0 ...

    """

    if trans is not None:
        ukn = listtoarray(trans.applystarlist(uknstars))
    else:
        ukn = listtoarray(uknstars)
    ref = listtoarray(refstars)

    dists = scipy.spatial.distance.cdist(ukn, ref)  # Big table of distances between ukn and ref
    mindists = np.min(dists, axis=1)  # For each ukn, the minimal distance
    minok = mindists <= r  # booleans for each ukn
    minokindexes = np.argwhere(minok).flatten()  # indexes of uknstars with matches

    if verbose:
        print("%i/%i stars with distance < r = %.1f (mean %.1f, median %.1f, std %.1f)" % (np.sum(minok),
                                                                                           len(uknstars), r,
                                                                                           np.mean(mindists[minok]),
                                                                                           np.median(mindists[minok]),
                                                                                           np.std(mindists[minok])))
    matchuknstars = []
    matchrefstars = []

    for i in minokindexes:  # we look for the second nearest ...
        sortedrefs = np.argsort(dists[i, :])
        firstdist = dists[i, sortedrefs[0]]
        seconddist = dists[i, sortedrefs[1]]
        if seconddist > 2.0*firstdist:  # Then the situation is clear, we keep it.
            matchuknstars.append(uknstars[i])
            matchrefstars.append(refstars[sortedrefs[0]])
        else:
            pass  # Then there is a companion, we skip it.

    if verbose:
        print("Filtered for companions, keeping %i/%i matches" % (len(matchuknstars), np.sum(minok)))

    if getstars is True:
        return (matchuknstars, matchrefstars)
    else:
        return len(matchuknstars)


In [ ]:
identify (a[0].uknmatchstars, a[0].refmatchstars )

# Run cell below

In [1]:
import tkinter.filedialog as fd
import json

file_path = fd.askopenfilename(title = "Choose the position list")
x_border = 400
y_border = 200
pos_file = open(file_path)
data = json.load(pos_file)
POS = data['POSITIONS']

labels = [P['LABEL'] for P in POS]
posX = [P['DEVICES'][0]['X'] for P in POS]
posY = [P['DEVICES'][0]['Y'] for P in POS]

upper_left_um_x = min(posX)
upper_left_um_y = max(posY)
file_path_coords = fd.askopenfilename(title = "Choose the coordinate .npy file", initialdir = os.path.dirname(file_path))
coords_t = np.load(file_path_coords)
n_t = int(len(coords_t)/2)
coords_t_seq = coords_t[:n_t,:]
coords_t_FRET = coords_t[n_t:,:]
[deltax, deltay] = np.mean(np.subtract(coords_t_seq,coords_t_FRET), axis = 0)
print(deltax,deltay)
print(upper_left_um_x, upper_left_um_y)


#X_c = [posX[i] for i,x in enumerate(labels) if labels[i]=='Pos287']
#Y_c = [posY[i] for i,x in enumerate(labels) if labels[i]=='Pos287']

labels_res = []
posX_res = []
posY_res = []

for i,x in enumerate(labels):
#     x = int(-850+(305-x_border)/2 + (posX[i] - 26384)/0.34) # tile 2 06/09/2022; 305 and 154 are the size of tfd 512 by 256 image
#     y = int(-385+(154-y_border)/2  - (posY[i] + 407.1)/0.34)  
#     x = int(-3967+(305-x_border)/2 + (posX[i] - 26384)/0.34) # tile 1 06/09/2022; 305 and 154 are the size of tfd 512 by 256 image
#     y = int(-405+(154-y_border)/2  - (posY[i] + 407.1)/0.34)   
#     x = int(1700+(330-x_border)/2 + (posX[i] - 27246.18)/0.34) # tile 2 19/07/2022
#     y = int(1571+(170-y_border)/2  - (posY[i] + 1149.66)/0.34)   
    x = int(deltax+(335-x_border)/2 + (posX[i] - upper_left_um_x)/0.34) # tile 2 06/09/2022;
    y = int(deltay+(120-y_border)/2  - (posY[i] - upper_left_um_y)/0.34)  # 305 and 154 are the size of tfd 512 by 256 image
#     x = int(2329+(330-x_border)/2 + (posX[i] - 28522.9)/0.34) # tile 1 19/07/2022
#     y = int(2042+(170-y_border)/2 - (posY[i] + 1313.3)/0.34) 
    if (x > 0 ) and (y > 0):
        if (y+y_border< 2944) and (x+x_border <2866) :
            labels_res.append (labels[i])
            posX_res.append (posX[i])
            posY_res.append( posY[i])


print (labels_res)
for pos in labels_res: 
    
    current_direct = "E:/MUSCLE data/220815_FC_Nano_Cas9_CCR5/Analysis_Misha/"
    path_ref = current_direct+ pos+ '/' + pos+ "_contr_MIN_seq_stack.tif"
    path_unk = current_direct + pos+ '/' + pos + "_beads_transformed.tiff"
    print(path_unk)
    a = run(path_ref, path_unk)
    
    uknmatchstars_coord = []
    refmatchstars_coord = []
    if len(a[0].refmatchstars) >= 3:
        for i in range (len(a[0].refmatchstars)):
            uknmatchstars_coord.append(a[0].uknmatchstars[i].x)
            uknmatchstars_coord.append(a[0].uknmatchstars[i].y)
            refmatchstars_coord.append(a[0].refmatchstars[i].x)
            refmatchstars_coord.append(a[0].refmatchstars[i].y)

        refmatchstars_coord = np.array (refmatchstars_coord)
        uknmatchstars_coord = np.array(uknmatchstars_coord)

        refmatchstars_coord = np.reshape ( refmatchstars_coord, ( int(len(a[0].refmatchstars)), 2 ) )
        uknmatchstars_coord = np.reshape ( uknmatchstars_coord, ( int(len(a[0].uknmatchstars)), 2 ) )
        result_coord = np.concatenate ((refmatchstars_coord,uknmatchstars_coord ), axis = 0)
        print (pos+"\n", result_coord )
        show_blob (path_unk, uknmatchstars_coord)
        show_blob (path_ref, refmatchstars_coord)
        np.save (current_direct+ pos+ '/' + pos+"matching_beads", result_coord)
    

importing Jupyter notebook from FRETseq.ipynb


C:\Users\panf\AppData\Roaming\Python\Python38\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


NameError: name 'current_direct' is not defined

In [ ]:
show_blob (path_unk, uknmatchstars_coord)
show_blob (path_ref, refmatchstars_coord)

In [ ]:
uknmatchstars_coord_x = []
refmatchstars_coord_x = []
uknmatchstars_coord_y = []
refmatchstars_coord_y = []
for i in range (len(a[0].refmatchstars)):
    uknmatchstars_coord_x.append(a[0].uknmatchstars[i].x)
    uknmatchstars_coord_y.append(a[0].uknmatchstars[i].y)
    refmatchstars_coord_x.append(a[0].refmatchstars[i].x)
    refmatchstars_coord_y.append(a[0].refmatchstars[i].y)
x_min = int(np.min(refmatchstars_coord_x))
x_max = int(np.max(refmatchstars_coord_x))
y_min = int(np.min(refmatchstars_coord_y))
y_max = int(np.max(refmatchstars_coord_y))
print(y_max-y_min)

In [ ]:
from PIL import Image
img1 = io.imread(path_ref) #reading the stack of images
                    #img1 = np.mean(img, axis = 0) #averaging by the stack
img1 = img1.astype("ushort") #turn from float format to ushort
img1 = img1 [y_min-100 : y_max+100 , x_min - 100: x_max+100] 
img1 = Image.fromarray (img1)
img1.save(current_direct + "\matching2.tif")